# JUDIAgent: Intelligent Workflow Automation for Seismic Modeling

**Authors:** Haoyun Li, Abhinav Prakash Gahlot, and Felix J. Herrmann, SLIM

---

## Tutorial Overview

This tutorial introduces **JUDIAgent**, an intelligent agentic automation framework built on the JUDI platform for PDE-constrained seismic modeling and inversion. 

The tutorial is divided into three parts:

1. **Part 1: Core Design of JUDI** - Learn how JUDI enables scalable forward and adjoint simulations using high-level abstractions for sources, receivers, and operators. We'll implement and differentiate seismic experiments on GPUs and CPUs.

2. **Part 2: JUDIAgent Framework** - Explore JUDIAgent, an agent-based system for automating acquisition setup and workflow orchestration. See how the agent configures survey geometries, boundary conditions, and experiment parameters, and manages modeling and inversion pipelines with minimal manual tuning.

3. **Part 3: Adaptive Experiment Configuration** - Demonstrate adaptive experiment configuration, where the agent responds to simulation feedback to refine inputs and improve computational efficiency.

Attendees will leave with hands-on experience using agentic tools to streamline and optimize seismic modeling workflows.


## Setup and Imports

First, let's set up our environment and import the necessary packages.

> **Important: Selecting Kernel**
> 
> This notebook uses a **Python kernel** because JUDIGPT is a Python package.
> 
> **How to select kernel:**
> 1. Click the "Select Kernel" button in the top right
> 2. Or press `Ctrl+Shift+P` (Windows/Linux) or `Cmd+Shift+P` (Mac), then type "Select Kernel"
> 3. Choose **Python 3** or your Python environment (if you have `.venv`, select the Python from `.venv`)
> 
> **Note:** This notebook will demonstrate both Python code (for JUDIGPT) and actual Julia code execution (for JUDI.jl). We'll use JUDIGPT's tools to execute Julia code directly.


In [2]:
# Python imports
import sys
import os
from pathlib import Path

# Import JUDIGPT as a package
from judigpt import agent, autonomous_agent
from judigpt.tools import retrieve_judi_examples, retrieve_function_documentation

print("JUDIGPT successfully imported!")
print(f"Agent available: {agent is not None}")
print(f"Autonomous agent available: {autonomous_agent is not None}")


JUDIGPT successfully imported!
Agent available: True
Autonomous agent available: True


## Part 1: Core Design of JUDI

In this section, we'll explore the core design of JUDI by implementing a basic seismic modeling example. We'll use materials from JUDI tutorials to understand the fundamental concepts.


### 1.1 Using JUDIGPT to Retrieve JUDI Examples

JUDIGPT provides tools to retrieve relevant JUDI.jl examples and documentation. Let's use the `retrieve_judi_examples` tool to find examples for basic 2D modeling.


In [3]:
# Use JUDIGPT's retrieval tool to find JUDI examples
# This demonstrates how to use JUDIGPT API programmatically
from langchain_core.runnables import RunnableConfig
from judigpt.configuration import BaseConfiguration

# Create a basic configuration
config = RunnableConfig(configurable={"agent_model": "default"})

# Retrieve examples for 2D seismic modeling
query = "2D seismic modeling with Model Geometry and judiVector"
examples = retrieve_judi_examples.invoke({"query": query}, config=config)

print("Retrieved JUDI examples:")
print("=" * 80)
print(examples[:2000])  # Print first 2000 characters
print("...")


╭─────────────────────────────────────── Retrieving from JUDI.jl examples ────────────────────────────────────────╮
│ Query: 2D seismic modeling with Model Geometry and judiVector                                                   │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

Retrieved JUDI examples:
# From `/localdata/hli853/JUDIGPT/src/judigpt/rag/judi/examples/scripts/modeling_medical_2D.jl`:
```julia
using JUDI, SegyIO, LinearAlgebra, PythonPlot
# Set up model structure
n = (121, 101)   # (x,y,z) or (x,z)
d = (2.5f0, 2.5f0) # in mm
o = (0., 0.)
# Velocity [km/s]
v = ones(Float32,n) .+ 0.5f0
v0 = ones(Float32,n) .+ 0.5f0
v[:,Int(round(end/2)):end] .= 4f0
# Slowness squared [s^2/km^2]
m = (1f0 ./ v).^2
m0 = (1f0 ./ v0).^2
dm = vec(m - m0)
# Setup model structure
nsrc = 1	# number of sources
model = Model(n, d, o, m)
model0 = Model(n, d, o, m0)
## Set up receiver geometry
nxrec = 120
xrec = range(d[1], stop=d[2]*(n[1]-1), length=nxrec)
yrec = 0f0
zrec = range(d[1], stop=d[1], length=nxrec)
# receiver sampling and recording time
timeR = 250f0   # receiver recording time [ms]
dtR = 0.25f0    # receiver sampling interval [ms]
# Set up receiver structure
recGeometry = Geometry(xrec, yrec, zrec; dt=dtR, t=timeR, nsrc=nsrc)
## Set up source geometry (cell array 

### 1.2 Basic JUDI.jl Setup

Now let's set up a basic JUDI.jl seismic modeling example. We'll use Julia through PyJulia or direct Julia execution.


In [4]:
# Julia code for basic JUDI setup
# This code will be executed using JUDI.jl

julia_code_basic = """
using JUDI, LinearAlgebra

# Set up model structure
# Grid parameters
n = (120, 100)   # (nx, nz) for 2D
d = (10.0, 10.0)  # Grid spacing in meters
o = (0.0, 0.0)    # Origin

# Create a simple velocity model
v = ones(Float32, n) .+ 0.5f0
v[:, Int(round(end/2)):end] .= 3.5f0

# Convert to squared slowness (required by JUDI)
m = (1f0 ./ v).^2

# Create Model structure
model = Model(n, d, o, m)

println("Model created successfully!")
println("Model shape: ", n)
println("Velocity range: ", extrema(v))
"""

print("Julia code prepared for basic JUDI setup:")
print(julia_code_basic)

# Now let's actually execute this Julia code using JUDIGPT's tools
print("\n" + "="*80)
print("Executing Julia code using JUDIGPT's run_julia_code tool...")
print("="*80)

from judigpt.tools import run_julia_code

# Execute the Julia code
result = run_julia_code.invoke({"code": julia_code_basic})
print("\nExecution result:")
print(result)


Julia code prepared for basic JUDI setup:

using JUDI, LinearAlgebra

# Set up model structure
# Grid parameters
n = (120, 100)   # (nx, nz) for 2D
d = (10.0, 10.0)  # Grid spacing in meters
o = (0.0, 0.0)    # Origin

# Create a simple velocity model
v = ones(Float32, n) .+ 0.5f0
v[:, Int(round(end/2)):end] .= 3.5f0

# Convert to squared slowness (required by JUDI)
m = (1f0 ./ v).^2

# Create Model structure
model = Model(n, d, o, m)

println("Model created successfully!")
println("Model shape: ", n)
println("Velocity range: ", extrema(v))


Executing Julia code using JUDIGPT's run_julia_code tool...


╭────────────────────────────────────────────────── Code Runner ──────────────────────────────────────────────────╮
│ Running code:                                                                                                   │
│                                                                                                                 │
│                                                                                                                 │
│                                                                                                                 │
│  using JUDI, LinearAlgebra                                                                                      │
│                                                                                                                 │
│  # Set up model structure                                                                                       │
│  # Grid parameters                                                                                              │
│  n = (120, 100)   # (nx, nz) for 2D                                                                             │
│  d = (10.0, 10.0)  # Grid spacing in meters                                                                     │
│  o = (0.0, 0.0)    # Origin                                                                                     │
│                                                                                                                 │
│  # Create a simple velocity model                                                                               │
│  v = ones(Float32, n) .+ 0.5f0                                                                                  │
│  v[:, Int(round(end/2)):end] .= 3.5f0                                                                           │
│                                                                                                                 │
│  # Convert to squared slowness (required by JUDI)                                                               │
│  m = (1f0 ./ v).^2                                                                                              │
│                                                                                                                 │
│  # Create Model structure                                                                                       │
│  model = Model(n, d, o, m)                                                                                      │
│                                                                                                                 │
│  println("Model created successfully!")                                                                         │
│  println("Model shape: ", n)                                                                                    │
│  println("Velocity range: "...                                                                                  │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭────────────────────────────────────────────── Code Runner Result ───────────────────────────────────────────────╮
│ Code succeeded in 5.06 seconds!                                                                                 │
│                                                                                                                 │
│ Output: Model created successfully! Model shape: (120, 100) Velocity range: (1.5f0, 3.5f0)                      │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯


Execution result:
Code executed successfully!


### 1.3 Using JUDIGPT Agent to Generate and Validate Code

Let's use JUDIGPT's autonomous agent to help us generate a complete seismic modeling workflow. The agent can retrieve examples, generate code, and validate it.


In [5]:
# Use JUDIGPT autonomous agent to generate a complete modeling example
# The agent will retrieve examples, generate code, and validate it

task_description = """
Create a complete 2D seismic modeling example using JUDI.jl that includes:
1. A velocity model with three layers
2. Source and receiver geometry setup
3. A Ricker wavelet source
4. Forward modeling to generate synthetic seismic data
5. Display the results
"""

print("Task for JUDIGPT agent:")
print(task_description)
print("\n" + "="*80)
print("\nYou can now invoke the agent with this task using:")
print("result = autonomous_agent_graph.invoke({")
print("    'messages': [{'role': 'user', 'content': task_description}]")
print("}, config=config)")


Task for JUDIGPT agent:

Create a complete 2D seismic modeling example using JUDI.jl that includes:
1. A velocity model with three layers
2. Source and receiver geometry setup
3. A Ricker wavelet source
4. Forward modeling to generate synthetic seismic data
5. Display the results



You can now invoke the agent with this task using:
result = autonomous_agent_graph.invoke({
    'messages': [{'role': 'user', 'content': task_description}]
}, config=config)


### 1.4 Complete Seismic Modeling Example

Based on JUDI tutorials, here's a complete example of 2D seismic modeling:


In [6]:
# Complete Julia code for 2D seismic modeling
# This is based on JUDI tutorial materials

julia_code_complete = """
using JUDI, LinearAlgebra

# ============================================
# 1. Create Model Structure
# ============================================
n = (120, 100)   # Grid size (nx, nz)
d = (10.0, 10.0)  # Grid spacing [m]
o = (0.0, 0.0)    # Origin [m]

# Create velocity model with three layers
v = ones(Float32, n) .+ 0.5f0
v[:, Int(round(end/3)):Int(round(2*end/3))] .= 2.0f0
v[:, Int(round(2*end/3)):end] .= 3.5f0

# Convert to squared slowness
m = (1f0 ./ v).^2

# Create Model
model = Model(n, d, o, m)

# ============================================
# 2. Create Acquisition Geometry
# ============================================
nsrc = 3  # Number of sources

# Receiver geometry
nxrec = 120
xrec = range(0f0, stop=(n[1]-1)*d[1], length=nxrec)
yrec = 0f0  # Must be set for 2D
zrec = range(d[2], stop=d[2], length=nxrec)

# Recording parameters
timeD = 1250f0   # Recording time [ms]
dtD = 2f0        # Sampling interval [ms]

# Create receiver geometry
recGeometry = Geometry(xrec, yrec, zrec; dt=dtD, t=timeD, nsrc=nsrc)

# Source geometry
xsrc = convertToCell(range(0f0, stop=(n[1]-1)*d[1], length=nsrc))
ysrc = convertToCell(range(0f0, stop=0f0, length=nsrc))
zsrc = convertToCell(range(d[2], stop=d[2], length=nsrc))

srcGeometry = Geometry(xsrc, ysrc, zsrc; dt=dtD, t=timeD)

# ============================================
# 3. Create Source Wavelet
# ============================================
f0 = 0.01f0  # Dominant frequency [kHz]
wavelet = ricker_wavelet(timeD, dtD, f0)
q = judiVector(srcGeometry, wavelet)

# ============================================
# 4. Setup Modeling Operator
# ============================================
opt = Options(subsampling_factor=2, space_order=16, free_surface=false)

# Create projection operators
Pr = judiProjection(recGeometry)
F = judiModeling(model; options=opt)
Ps = judiProjection(srcGeometry)

# ============================================
# 5. Forward Modeling
# ============================================
dobs = Pr*F*adjoint(Ps)*q

println("Forward modeling completed!")
println("Number of shots: ", nsrc)
println("Data size for shot 1: ", size(dobs[1].data))
"""

print("Complete Julia code for 2D seismic modeling:")
print(julia_code_complete)

# Execute the complete modeling example
print("\n" + "="*80)
print("Executing complete 2D seismic modeling example...")
print("="*80)
print("Note: This may take a moment as JUDI loads and compiles...")

from judigpt.tools import run_julia_code

# Execute the complete Julia code
result = run_julia_code.invoke({"code": julia_code_complete})
print("\nExecution result:")
print(result)


Complete Julia code for 2D seismic modeling:

using JUDI, LinearAlgebra

# ============================================
# 1. Create Model Structure
# ============================================
n = (120, 100)   # Grid size (nx, nz)
d = (10.0, 10.0)  # Grid spacing [m]
o = (0.0, 0.0)    # Origin [m]

# Create velocity model with three layers
v = ones(Float32, n) .+ 0.5f0
v[:, Int(round(end/3)):Int(round(2*end/3))] .= 2.0f0
v[:, Int(round(2*end/3)):end] .= 3.5f0

# Convert to squared slowness
m = (1f0 ./ v).^2

# Create Model
model = Model(n, d, o, m)

# ============================================
# 2. Create Acquisition Geometry
# ============================================
nsrc = 3  # Number of sources

# Receiver geometry
nxrec = 120
xrec = range(0f0, stop=(n[1]-1)*d[1], length=nxrec)
yrec = 0f0  # Must be set for 2D
zrec = range(d[2], stop=d[2], length=nxrec)

# Recording parameters
timeD = 1250f0   # Recording time [ms]
dtD = 2f0        # Sampling interval [ms]

# Create receiver

╭────────────────────────────────────────────────── Code Runner ──────────────────────────────────────────────────╮
│ Running code:                                                                                                   │
│                                                                                                                 │
│                                                                                                                 │
│                                                                                                                 │
│  using JUDI, LinearAlgebra                                                                                      │
│                                                                                                                 │
│  # ============================================                                                                 │
│  # 1. Create Model Structure                                                                                    │
│  # ============================================                                                                 │
│  n = (120, 100)   # Grid size (nx, nz)                                                                          │
│  d = (10.0, 10.0)  # Grid spacing [m]                                                                           │
│  o = (0.0, 0.0)    # Origin [m]                                                                                 │
│                                                                                                                 │
│  # Create velocity model with three layers                                                                      │
│  v = ones(Float32, n) .+ 0.5f0                                                                                  │
│  v[:, Int(round(end/3)):Int(round(2*end/3))] .= 2.0f0                                                           │
│  v[:, Int(round(2*end/3)):end] .= 3.5f0                                                                         │
│                                                                                                                 │
│  # Convert to squared slowness                                                                                  │
│  m = (1f0 ./ v).^2                                                                                              │
│                                                                                                                 │
│  # Create Model                                                                                                 │
│  model...                                                                                                       │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭────────────────────────────────────────────── Code Runner Result ───────────────────────────────────────────────╮
│ Code succeeded in 18.25 seconds!                                                                                │
│                                                                                                                 │
│ Output: Forward modeling completed! Number of shots: 3 Data size for shot 1: (1,)                               │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯


Execution result:
Code executed successfully!


### 1.5 Running a Simple Forward Modeling Example

Let's execute a simplified JUDI forward modeling example to see it in action:


In [7]:
# A simplified forward modeling example that we'll actually execute
# This demonstrates a minimal working JUDI example

simple_modeling_code = """
using JUDI, LinearAlgebra

# Create a small 2D model for quick execution
n = (60, 50)   # Smaller grid for faster execution
d = (10.0, 10.0)
o = (0.0, 0.0)

# Simple two-layer velocity model
v = ones(Float32, n) .* 1.5f0
v[:, Int(round(end/2)):end] .= 2.5f0

# Convert to squared slowness
m = (1f0 ./ v).^2
model = Model(n, d, o, m)

# Simple acquisition: 1 source, receivers along surface
nsrc = 1
xrec = range(0f0, stop=(n[1]-1)*d[1], length=60)
yrec = 0f0
zrec = range(d[2], stop=d[2], length=60)

timeD = 500f0   # Shorter recording time for faster execution
dtD = 2f0

recGeometry = Geometry(xrec, yrec, zrec; dt=dtD, t=timeD, nsrc=nsrc)

xsrc = convertToCell([(n[1]-1)*d[1]/2])
ysrc = convertToCell([0f0])
zsrc = convertToCell([d[2]])

srcGeometry = Geometry(xsrc, ysrc, zsrc; dt=dtD, t=timeD)

# Create Ricker wavelet
f0 = 0.015f0
wavelet = ricker_wavelet(timeD, dtD, f0)
q = judiVector(srcGeometry, wavelet)

# Setup modeling operator with reduced space_order for faster execution
opt = Options(subsampling_factor=4, space_order=8, free_surface=false)

Pr = judiProjection(recGeometry)
F = judiModeling(model; options=opt)
Ps = judiProjection(srcGeometry)

# Perform forward modeling
println("Starting forward modeling...")
dobs = Pr*F*adjoint(Ps)*q

println("\\nForward modeling completed successfully!")
println("Model grid size: ", n)
println("Number of sources: ", nsrc)
println("Number of receivers: ", length(xrec))
println("Recording time: ", timeD, " ms")
println("Data shape for shot 1: ", size(dobs[1].data))
println("Data time samples: ", size(dobs[1].data, 1))
println("Data receiver channels: ", size(dobs[1].data, 2))
"""

print("Simple forward modeling example:")
print("="*80)
print(simple_modeling_code)
print("\n" + "="*80)
print("Executing simplified JUDI forward modeling...")
print("(This may take 30-60 seconds as JUDI loads and compiles)")
print("="*80)

from judigpt.tools import run_julia_code

# Execute the simplified modeling code
result = run_julia_code.invoke({"code": simple_modeling_code})
print("\nExecution result:")
print(result)


Simple forward modeling example:

using JUDI, LinearAlgebra

# Create a small 2D model for quick execution
n = (60, 50)   # Smaller grid for faster execution
d = (10.0, 10.0)
o = (0.0, 0.0)

# Simple two-layer velocity model
v = ones(Float32, n) .* 1.5f0
v[:, Int(round(end/2)):end] .= 2.5f0

# Convert to squared slowness
m = (1f0 ./ v).^2
model = Model(n, d, o, m)

# Simple acquisition: 1 source, receivers along surface
nsrc = 1
xrec = range(0f0, stop=(n[1]-1)*d[1], length=60)
yrec = 0f0
zrec = range(d[2], stop=d[2], length=60)

timeD = 500f0   # Shorter recording time for faster execution
dtD = 2f0

recGeometry = Geometry(xrec, yrec, zrec; dt=dtD, t=timeD, nsrc=nsrc)

xsrc = convertToCell([(n[1]-1)*d[1]/2])
ysrc = convertToCell([0f0])
zsrc = convertToCell([d[2]])

srcGeometry = Geometry(xsrc, ysrc, zsrc; dt=dtD, t=timeD)

# Create Ricker wavelet
f0 = 0.015f0
wavelet = ricker_wavelet(timeD, dtD, f0)
q = judiVector(srcGeometry, wavelet)

# Setup modeling operator with reduced space_orde

╭────────────────────────────────────────────────── Code Runner ──────────────────────────────────────────────────╮
│ Running code:                                                                                                   │
│                                                                                                                 │
│                                                                                                                 │
│                                                                                                                 │
│  using JUDI, LinearAlgebra                                                                                      │
│                                                                                                                 │
│  # Create a small 2D model for quick execution                                                                  │
│  n = (60, 50)   # Smaller grid for faster execution                                                             │
│  d = (10.0, 10.0)                                                                                               │
│  o = (0.0, 0.0)                                                                                                 │
│                                                                                                                 │
│  # Simple two-layer velocity model                                                                              │
│  v = ones(Float32, n) .* 1.5f0                                                                                  │
│  v[:, Int(round(end/2)):end] .= 2.5f0                                                                           │
│                                                                                                                 │
│  # Convert to squared slowness                                                                                  │
│  m = (1f0 ./ v).^2                                                                                              │
│  model = Model(n, d, o, m)                                                                                      │
│                                                                                                                 │
│  # Simple acquisition: 1 source, receivers along surface                                                        │
│  nsrc = 1                                                                                                       │
│  xrec = range(0f0, stop=(n[1]-1)*d[1], length=60)                                                               │
│  yrec = 0f0                                                                                                     │
│  zrec = range(d[2], stop=d[2], l...                                                                             │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭────────────────────────────────────────────── Code Runner Result ───────────────────────────────────────────────╮
│ Code failed!                                                                                                    │
│                                                                                                                 │
│ ERROR: MethodError: no method matching GeometryIC{Float64}(::Vector{Vector{Float64}},                           │
│ ::Vector{Vector{Float32}}, ::Vector{Vector{Float64}}, ::Vector{StepRangeLen{Float64, Float64, Float64, Int64}}) │
│ The type GeometryIC{Float64} exists, but no method is defined for this combination of argument types when       │
│ trying to construct it.                                                                                         │
│                                                                                                                 │
│ Closest candidates are: GeometryIC{T}(::Union{Vector{T}, Array{Vector{T}, 1}}, !Matched::Union{Vector{T},       │
│ Array{Vector{T}, 1}}, ::Union{Vector{T}, Array{Vector{T}, 1}}, ::Vector{<:StepRangeLen{T}}) where T @ JUDI      │
│ ~/.julia/packages/JUDI/fp1uQ/src/TimeModeling/Types/GeometryStructure.jl:31 GeometryIC{T}(::Union{Vector{T},    │
│ Array{Vector{T}, 1}}, !Matched::Union{Vector{T}, Array{Vector{T}, 1}}, ::Union{Vector{T}, Array{Vector{T}, 1}}, │
│ !Matched::Vector{T}, !Matched::Vector{<:Integer}, !Matched::Vector{T}) where T @ JUDI                           │
│ ~/.julia/packages/JUDI/fp1uQ/src/TimeModeling/Types/GeometryStructure.jl:26                                     │
│                                                                                                                 │
│ Stacktrace: [1] Geometry(xloc::Vector{Vector{Float64}}, yloc::Vector{Vector{Float32}},                          │
│ zloc::Vector{Vector{Float64}}; dt::Float32, t::Float32, t0::Int64) @ JUDI                                       │
│ ~/.julia/packages/JUDI/fp1uQ/src/TimeModeling/Types/GeometryStructure.jl:199 [2] top-level scope @ none:32      │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯


Execution result:
## Code runner error:
Running the code generated failed with the following error:
ERROR: MethodError: no method matching GeometryIC{Float64}(::Vector{Vector{Float64}}, ::Vector{Vector{Float32}}, ::Vector{Vector{Float64}}, ::Vector{StepRangeLen{Float64, Float64, Float64, Int64}})
The type `GeometryIC{Float64}` exists, but no method is defined for this combination of argument types when trying to construct it.

Closest candidates are:
  GeometryIC{T}(::Union{Vector{T}, Array{Vector{T}, 1}}, !Matched::Union{Vector{T}, Array{Vector{T}, 1}}, ::Union{Vector{T}, Array{Vector{T}, 1}}, ::Vector{<:StepRangeLen{T}}) where T
   @ JUDI ~/.julia/packages/JUDI/fp1uQ/src/TimeModeling/Types/GeometryStructure.jl:31
  GeometryIC{T}(::Union{Vector{T}, Array{Vector{T}, 1}}, !Matched::Union{Vector{T}, Array{Vector{T}, 1}}, ::Union{Vector{T}, Array{Vector{T}, 1}}, !Matched::Vector{T}, !Matched::Vector{<:Integer}, !Matched::Vector{T}) where T
   @ JUDI ~/.julia/packages/JUDI/fp1uQ/src/TimeM

## Part 2: JUDIAgent Framework

In this section, we'll explore how JUDIAgent can automate workflow orchestration and acquisition setup.


### 2.1 Using JUDIGPT Agent for Automated Code Generation

JUDIGPT's agent can automatically retrieve relevant examples, understand requirements, and generate optimized code. Let's demonstrate this:


### 2.2 Workflow Orchestration Example

JUDIAgent can manage complex modeling and inversion pipelines. Here's an example workflow:


### 2.3 Automated Boundary Condition Configuration

JUDIAgent can automatically configure boundary conditions and simulation parameters:


## Part 3: Adaptive Experiment Configuration

In this final section, we'll demonstrate how JUDIAgent can adaptively refine experiment configurations based on simulation feedback.


### 3.1 Adaptive Parameter Tuning

JUDIAgent can respond to simulation feedback to improve computational efficiency:


### 3.2 Feedback-Driven Workflow Refinement

Demonstrate how JUDIAgent can refine workflows based on intermediate results:


### 3.3 Complete Adaptive Workflow Example

Putting it all together - a complete example of adaptive experiment configuration:


### 2.1 Using JUDIGPT Agent for Automated Code Generation

JUDIGPT's agent can automatically retrieve relevant examples, understand requirements, and generate optimized code. Let's demonstrate this:


In [8]:
# Example: Use JUDIGPT agent to automatically configure a survey geometry
# The agent will retrieve examples and generate appropriate code

survey_task = """
Create a function that sets up a marine seismic survey geometry with:
- 10 sources evenly spaced along a line
- Receivers in a streamer configuration
- Appropriate depth settings for marine acquisition
- Return both source and receiver Geometry objects
"""

print("Survey configuration task:")
print(survey_task)
print("\nThis task can be given to the JUDIGPT agent to automatically generate the code.")


Survey configuration task:

Create a function that sets up a marine seismic survey geometry with:
- 10 sources evenly spaced along a line
- Receivers in a streamer configuration
- Appropriate depth settings for marine acquisition
- Return both source and receiver Geometry objects


This task can be given to the JUDIGPT agent to automatically generate the code.


### 2.2 Workflow Orchestration Example

JUDIAgent can manage complex modeling and inversion pipelines. Here's an example workflow:


In [9]:
# Python code demonstrating workflow orchestration with JUDIGPT

def orchestrate_seismic_workflow(model_params, acquisition_params, inversion_params):
    """
    Orchestrate a complete seismic modeling and inversion workflow.
    
    This function demonstrates how JUDIAgent can manage a pipeline:
    1. Model setup
    2. Acquisition configuration
    3. Forward modeling
    4. Inversion setup
    5. Optimization
    """
    workflow_steps = []
    
    # Step 1: Use JUDIGPT to retrieve relevant examples for model setup
    model_query = f"Create Model with shape {model_params['shape']}, spacing {model_params['spacing']}"
    workflow_steps.append({
        'step': 'Model Setup',
        'query': model_query,
        'description': 'Retrieve and apply model setup examples'
    })
    
    # Step 2: Configure acquisition geometry
    geom_query = f"Setup Geometry for {acquisition_params['nsrc']} sources, {acquisition_params['nrec']} receivers"
    workflow_steps.append({
        'step': 'Acquisition Setup',
        'query': geom_query,
        'description': 'Automatically configure survey geometry'
    })
    
    # Step 3: Forward modeling
    modeling_query = "Forward modeling with judiModeling operator"
    workflow_steps.append({
        'step': 'Forward Modeling',
        'query': modeling_query,
        'description': 'Generate synthetic seismic data'
    })
    
    # Step 4: Inversion setup
    inversion_query = f"Setup FWI inversion with {inversion_params['method']} optimization"
    workflow_steps.append({
        'step': 'Inversion Setup',
        'query': inversion_query,
        'description': 'Configure inversion parameters and objective function'
    })
    
    return workflow_steps

# Example usage
model_params = {
    'shape': (200, 150),
    'spacing': (10.0, 10.0),
    'origin': (0.0, 0.0)
}

acquisition_params = {
    'nsrc': 20,
    'nrec': 200,
    'depth': 50.0
}

inversion_params = {
    'method': 'L-BFGS',
    'max_iterations': 50
}

workflow = orchestrate_seismic_workflow(model_params, acquisition_params, inversion_params)

print("Workflow Orchestration Plan:")
print("=" * 80)
for i, step in enumerate(workflow, 1):
    print(f"\nStep {i}: {step['step']}")
    print(f"  Query: {step['query']}")
    print(f"  Description: {step['description']}")


Workflow Orchestration Plan:

Step 1: Model Setup
  Query: Create Model with shape (200, 150), spacing (10.0, 10.0)
  Description: Retrieve and apply model setup examples

Step 2: Acquisition Setup
  Query: Setup Geometry for 20 sources, 200 receivers
  Description: Automatically configure survey geometry

Step 3: Forward Modeling
  Query: Forward modeling with judiModeling operator
  Description: Generate synthetic seismic data

Step 4: Inversion Setup
  Query: Setup FWI inversion with L-BFGS optimization
  Description: Configure inversion parameters and objective function


### 2.3 Automated Boundary Condition Configuration

JUDIAgent can automatically configure boundary conditions and simulation parameters:


In [10]:
# Example: Automated Options configuration

def configure_simulation_options(model_type="acoustic", free_surface=True, gpu=False):
    """
    Automatically configure JUDI Options based on simulation requirements.
    
    This demonstrates how JUDIAgent can intelligently set up simulation
    parameters based on the problem type.
    """
    
    # Use JUDIGPT to retrieve best practices for options configuration
    options_query = f"JUDI Options configuration for {model_type} modeling with free_surface={free_surface}"
    
    # Base configuration
    base_options = {
        'space_order': 16,
        'free_surface': free_surface,
        'subsampling_factor': 2,
        'dt_comp': None,  # Will be computed automatically
    }
    
    # Adjust based on model type
    if model_type == "elastic":
        base_options['space_order'] = 20  # Higher order for elastic
    
    # GPU-specific settings
    if gpu:
        base_options['checkpointing'] = True
        base_options['limit_m'] = True
    
    return base_options, options_query

# Example configurations
acoustic_opts, query1 = configure_simulation_options("acoustic", free_surface=True)
elastic_opts, query2 = configure_simulation_options("elastic", free_surface=False, gpu=True)

print("Acoustic Configuration:")
print(acoustic_opts)
print(f"\nQuery for JUDIGPT: {query1}")
print("\n" + "="*80)
print("\nElastic GPU Configuration:")
print(elastic_opts)
print(f"\nQuery for JUDIGPT: {query2}")


Acoustic Configuration:
{'space_order': 16, 'free_surface': True, 'subsampling_factor': 2, 'dt_comp': None}

Query for JUDIGPT: JUDI Options configuration for acoustic modeling with free_surface=True


Elastic GPU Configuration:
{'space_order': 20, 'free_surface': False, 'subsampling_factor': 2, 'dt_comp': None, 'checkpointing': True, 'limit_m': True}

Query for JUDIGPT: JUDI Options configuration for elastic modeling with free_surface=False


## Part 3: Adaptive Experiment Configuration

In this final section, we'll demonstrate how JUDIAgent can adaptively refine experiment configurations based on simulation feedback.


### 3.1 Adaptive Parameter Tuning

JUDIAgent can respond to simulation feedback to improve computational efficiency:


In [11]:
# Example: Adaptive configuration based on simulation results

class AdaptiveExperimentConfig:
    """
    Demonstrates adaptive experiment configuration using JUDIAgent.
    The agent can refine parameters based on simulation feedback.
    """
    
    def __init__(self, initial_config):
        self.config = initial_config
        self.history = []
    
    def run_simulation(self, model, geometry, source):
        """
        Run simulation and collect performance metrics.
        """
        # Simulate running a JUDI simulation
        # In practice, this would call actual JUDI.jl code
        
        metrics = {
            'computation_time': 120.5,  # seconds
            'memory_usage': 8.2,  # GB
            'stability': 'stable',
            'accuracy': 0.95
        }
        
        return metrics
    
    def adapt_configuration(self, metrics, target_time=100.0):
        """
        Use JUDIGPT agent to suggest configuration improvements.
        """
        
        # If computation is too slow, ask JUDIGPT for optimization suggestions
        if metrics['computation_time'] > target_time:
            optimization_query = f"""
            Optimize JUDI simulation configuration:
            - Current computation time: {metrics['computation_time']}s
            - Target: {target_time}s
            - Memory usage: {metrics['memory_usage']}GB
            - Current space_order: {self.config.get('space_order', 'default')}
            - Current subsampling_factor: {self.config.get('subsampling_factor', 'default')}
            
            Suggest optimizations to reduce computation time while maintaining accuracy.
            """
            
            # In practice, this would call the JUDIGPT agent
            suggested_changes = {
                'subsampling_factor': self.config.get('subsampling_factor', 2) + 1,
                'space_order': max(8, self.config.get('space_order', 16) - 2)
            }
            
            self.config.update(suggested_changes)
            self.history.append({
                'iteration': len(self.history) + 1,
                'metrics': metrics,
                'changes': suggested_changes,
                'query': optimization_query
            })
            
            return suggested_changes, optimization_query
        
        return None, None
    
    def get_optimization_history(self):
        """Return the history of adaptive optimizations."""
        return self.history

# Example usage
initial_config = {
    'space_order': 16,
    'subsampling_factor': 2,
    'free_surface': True
}

adaptive_config = AdaptiveExperimentConfig(initial_config)

# Simulate running a simulation and adapting
metrics = adaptive_config.run_simulation(None, None, None)
changes, query = adaptive_config.adapt_configuration(metrics, target_time=100.0)

print("Initial Configuration:")
print(initial_config)
print("\nSimulation Metrics:")
print(metrics)
print("\nSuggested Changes:")
print(changes)
print("\nOptimization Query for JUDIGPT:")
print(query)
print("\nUpdated Configuration:")
print(adaptive_config.config)


Initial Configuration:
{'space_order': 14, 'subsampling_factor': 3, 'free_surface': True}

Simulation Metrics:
{'computation_time': 120.5, 'memory_usage': 8.2, 'stability': 'stable', 'accuracy': 0.95}

Suggested Changes:
{'subsampling_factor': 3, 'space_order': 14}

Optimization Query for JUDIGPT:

            Optimize JUDI simulation configuration:
            - Current computation time: 120.5s
            - Target: 100.0s
            - Memory usage: 8.2GB
            - Current space_order: 16
            - Current subsampling_factor: 2

            Suggest optimizations to reduce computation time while maintaining accuracy.
            

Updated Configuration:
{'space_order': 14, 'subsampling_factor': 3, 'free_surface': True}


### 3.2 Feedback-Driven Workflow Refinement

Demonstrate how JUDIAgent can refine workflows based on intermediate results:


In [12]:
# Example: Feedback-driven workflow refinement

def refine_workflow_with_feedback(initial_workflow, simulation_results):
    """
    Use JUDIGPT agent to refine workflow based on simulation feedback.
    """
    
    # Analyze simulation results
    issues = []
    if simulation_results.get('convergence_rate', 1.0) < 0.5:
        issues.append("slow_convergence")
    if simulation_results.get('memory_usage', 0) > 16.0:
        issues.append("high_memory")
    if simulation_results.get('numerical_stability', 'stable') != 'stable':
        issues.append("instability")
    
    # Generate queries for JUDIGPT agent based on issues
    refinement_queries = []
    
    if 'slow_convergence' in issues:
        query = """
        Optimize FWI inversion workflow for faster convergence:
        - Current convergence rate is slow
        - Suggest preconditioning strategies
        - Recommend optimization algorithm adjustments
        """
        refinement_queries.append({
            'issue': 'slow_convergence',
            'query': query,
            'suggested_action': 'Retrieve FWI optimization examples'
        })
    
    if 'high_memory' in issues:
        query = """
        Reduce memory usage in JUDI simulation:
        - Current memory usage exceeds 16GB
        - Suggest checkpointing strategies
        - Recommend subsampling or model compression techniques
        """
        refinement_queries.append({
            'issue': 'high_memory',
            'query': query,
            'suggested_action': 'Retrieve memory optimization examples'
        })
    
    if 'instability' in issues:
        query = """
        Fix numerical instability in JUDI simulation:
        - Simulation shows numerical instability
        - Suggest CFL condition adjustments
        - Recommend space_order or time stepping modifications
        """
        refinement_queries.append({
            'issue': 'instability',
            'query': query,
            'suggested_action': 'Retrieve stability and CFL examples'
        })
    
    return refinement_queries

# Example: Simulate workflow refinement
initial_workflow = {
    'steps': ['model_setup', 'forward_modeling', 'inversion'],
    'config': {'space_order': 16, 'subsampling_factor': 2}
}

simulation_results = {
    'convergence_rate': 0.3,
    'memory_usage': 18.5,
    'numerical_stability': 'stable'
}

refinements = refine_workflow_with_feedback(initial_workflow, simulation_results)

print("Workflow Refinement Analysis:")
print("=" * 80)
print("\nSimulation Results:")
for key, value in simulation_results.items():
    print(f"  {key}: {value}")

print("\nSuggested Refinements:")
for i, refinement in enumerate(refinements, 1):
    print(f"\n{i}. Issue: {refinement['issue']}")
    print(f"   Action: {refinement['suggested_action']}")
    print(f"   Query: {refinement['query'].strip()[:100]}...")


Workflow Refinement Analysis:

Simulation Results:
  convergence_rate: 0.3
  memory_usage: 18.5
  numerical_stability: stable

Suggested Refinements:

1. Issue: slow_convergence
   Action: Retrieve FWI optimization examples
   Query: Optimize FWI inversion workflow for faster convergence:
        - Current convergence rate is slow
 ...

2. Issue: high_memory
   Action: Retrieve memory optimization examples
   Query: Reduce memory usage in JUDI simulation:
        - Current memory usage exceeds 16GB
        - Sugges...


In [13]:
# Complete adaptive workflow example

def adaptive_seismic_experiment(problem_spec, max_iterations=5):
    """
    Run an adaptive seismic experiment that refines configuration based on feedback.
    
    This demonstrates the full power of JUDIAgent:
    1. Initial setup using JUDIGPT to retrieve best practices
    2. Run simulation
    3. Analyze results
    4. Use JUDIGPT to suggest improvements
    5. Iterate until convergence
    """
    
    iteration_history = []
    
    # Initial setup query for JUDIGPT
    setup_query = f"""
    Setup a {problem_spec['type']} seismic modeling experiment:
    - Model size: {problem_spec['model_size']}
    - Number of sources: {problem_spec['nsrc']}
    - Target: {problem_spec['target']}
    
    Provide optimized configuration for this problem.
    """
    
    # Initial configuration (would come from JUDIGPT agent)
    current_config = {
        'space_order': 16,
        'subsampling_factor': 2,
        'free_surface': problem_spec.get('free_surface', True)
    }
    
    for iteration in range(max_iterations):
        # Simulate running experiment (in practice, this would run actual JUDI code)
        results = {
            'computation_time': 150.0 - iteration * 10.0,  # Simulated improvement
            'memory_usage': 12.0 - iteration * 0.5,
            'accuracy': 0.90 + iteration * 0.02,
            'converged': iteration >= 3
        }
        
        iteration_history.append({
            'iteration': iteration + 1,
            'config': current_config.copy(),
            'results': results
        })
        
        # Check if we should adapt
        if results['converged']:
            break
        
        # Generate adaptation query for JUDIGPT
        if results['computation_time'] > problem_spec.get('target_time', 100.0):
            adapt_query = f"""
            Iteration {iteration + 1} results:
            - Computation time: {results['computation_time']}s (target: {problem_spec.get('target_time', 100.0)}s)
            - Memory: {results['memory_usage']}GB
            - Accuracy: {results['accuracy']}
            
            Current config: {current_config}
            
            Suggest configuration changes to reduce computation time while maintaining accuracy.
            """
            
            # Simulated adaptation (in practice, JUDIGPT agent would suggest this)
            if iteration < 2:
                current_config['subsampling_factor'] += 1
            else:
                current_config['space_order'] = max(8, current_config['space_order'] - 2)
    
    return iteration_history, setup_query

# Run adaptive experiment
problem_spec = {
    'type': '2D acoustic FWI',
    'model_size': (200, 150),
    'nsrc': 20,
    'target': 'velocity inversion',
    'target_time': 100.0,
    'free_surface': True
}

history, setup_query = adaptive_seismic_experiment(problem_spec)

print("Adaptive Seismic Experiment Results:")
print("=" * 80)
print("\nInitial Setup Query for JUDIGPT:")
print(setup_query)
print("\n" + "=" * 80)
print("\nIteration History:")
for entry in history:
    print(f"\nIteration {entry['iteration']}:")
    print(f"  Config: {entry['config']}")
    print(f"  Results: Time={entry['results']['computation_time']:.1f}s, ")
    print(f"           Memory={entry['results']['memory_usage']:.1f}GB, ")
    print(f"           Accuracy={entry['results']['accuracy']:.2f}")
    if entry['results']['converged']:
        print("  ✓ Converged!")


Adaptive Seismic Experiment Results:

Initial Setup Query for JUDIGPT:

    Setup a 2D acoustic FWI seismic modeling experiment:
    - Model size: (200, 150)
    - Number of sources: 20
    - Target: velocity inversion

    Provide optimized configuration for this problem.
    


Iteration History:

Iteration 1:
  Config: {'space_order': 16, 'subsampling_factor': 2, 'free_surface': True}
  Results: Time=150.0s, 
           Memory=12.0GB, 
           Accuracy=0.90

Iteration 2:
  Config: {'space_order': 16, 'subsampling_factor': 3, 'free_surface': True}
  Results: Time=140.0s, 
           Memory=11.5GB, 
           Accuracy=0.92

Iteration 3:
  Config: {'space_order': 16, 'subsampling_factor': 4, 'free_surface': True}
  Results: Time=130.0s, 
           Memory=11.0GB, 
           Accuracy=0.94

Iteration 4:
  Config: {'space_order': 14, 'subsampling_factor': 4, 'free_surface': True}
  Results: Time=120.0s, 
           Memory=10.5GB, 
           Accuracy=0.96
  ✓ Converged!


## Summary and Next Steps

This tutorial has demonstrated:

1. **Core JUDI Design**: How to use JUDI.jl for seismic modeling with high-level abstractions
2. **JUDIAgent Framework**: How to use JUDIGPT as a package to automate workflow orchestration
3. **Adaptive Configuration**: How JUDIAgent can refine experiments based on feedback

### Key Takeaways:

- **JUDI.jl** provides powerful abstractions (`Model`, `Geometry`, `judiVector`, `judiModeling`) for seismic modeling
- **JUDIGPT** can be used as a Python package to access intelligent code generation and retrieval capabilities
- **JUDIAgent** enables automated workflow orchestration with minimal manual tuning
- **Adaptive configuration** allows the system to improve efficiency based on simulation feedback

### Next Steps:

1. Explore more JUDI.jl examples using `retrieve_judi_examples`
2. Use the `autonomous_agent` for complex code generation tasks
3. Build custom workflows that leverage JUDIGPT's retrieval and generation capabilities
4. Experiment with adaptive configuration for your specific use cases

### Resources:

- JUDI.jl Documentation: Available through JUDIGPT's retrieval tools
- JUDI Examples: Use `retrieve_judi_examples` to find relevant examples
- JUDIGPT API: Import `agent` or `autonomous_agent` from `judigpt` package


In [14]:
# Final demonstration: Quick reference for using JUDIGPT

print("JUDIGPT Quick Reference:")
print("=" * 80)
print("\n1. Import JUDIGPT:")
print("   from judigpt import agent, autonomous_agent")
print("   from judigpt.tools import retrieve_judi_examples")

print("\n2. Retrieve JUDI examples:")
print("   examples = retrieve_judi_examples.invoke({")
print("       'query': 'your search query'")
print("   }, config=config)")

print("\n3. Use autonomous agent for code generation:")
print("   result = autonomous_agent_graph.invoke({")
print("       'messages': [{'role': 'user', 'content': 'your task'}]")
print("   }, config=config)")

print("\n4. Use standard agent for interactive assistance:")
print("   result = agent_graph.invoke({")
print("       'messages': [{'role': 'user', 'content': 'your question'}]")
print("   }, config=config)")

print("\n" + "=" * 80)
print("\nTutorial completed! Explore JUDI.jl and JUDIGPT to build your workflows.")


JUDIGPT Quick Reference:

1. Import JUDIGPT:
   from judigpt import agent, autonomous_agent
   from judigpt.tools import retrieve_judi_examples

2. Retrieve JUDI examples:
   examples = retrieve_judi_examples.invoke({
       'query': 'your search query'
   }, config=config)

3. Use autonomous agent for code generation:
   result = autonomous_agent_graph.invoke({
       'messages': [{'role': 'user', 'content': 'your task'}]
   }, config=config)

4. Use standard agent for interactive assistance:
   result = agent_graph.invoke({
       'messages': [{'role': 'user', 'content': 'your question'}]
   }, config=config)


Tutorial completed! Explore JUDI.jl and JUDIGPT to build your workflows.
